# Point Prediction

In [11]:
using CSV, DataFrames, Plots, Random, Statistics, LinearAlgebra, JuMP, Gurobi, StatsBase, Dates
using Flux: onehotbatch

In [12]:
# Load datasets
daily_sales = CSV.read("modelling data/daily_sales.csv", DataFrame)
daily_sales_ingredients_predictions = CSV.read("modelling data/daily_sales_ingredients_predictions.csv", DataFrame)
daily_sales_ingredients_actuals = CSV.read("modelling data/daily_sales_ingredients_actuals.csv", DataFrame)
daily_sales_actuals = CSV.read("modelling data/daily_sales_actuals.csv", DataFrame)
ingredient_prices = CSV.read("modelling data/ingredient_prices.csv", DataFrame)
pizza_ingredient_matrix = CSV.read("modelling data/pizza_ingredient_matrix.csv", DataFrame)
pizzas = CSV.read("modelling data/pizzas.csv", DataFrame)

# Convert DataFrames to matrices if needed
daily_sales_mat = Matrix(daily_sales[:, Not(:date)])   # remove 'date' column
ingredient_prices_mat = Matrix(ingredient_prices[:, Not(:ingredient)]) # keep only prices
pizza_ingredient_matrix_mat = Matrix(pizza_ingredient_matrix[:, Not([:pizza_type_id, :pizza_name, :category])])
pizzas_mat = Matrix(pizzas[:, Not([:pizza_id, :pizza_type_id, :size])]) # only prices

# Get original column names
orig_cols = names(daily_sales_ingredients_predictions)

# Create cleaned names by removing the prefix where it appears
clean_cols = Symbol.(replace.(String.(orig_cols), "ingredient_" => ""))

# Apply new names back to the DataFrame
rename!(daily_sales_ingredients_predictions, Dict(orig_cols .=> clean_cols))
rename!(daily_sales_ingredients_actuals, Dict(orig_cols .=> clean_cols));

In [13]:
df = DataFrame(CSV.File("modelling data/daily_sales_ingredients.csv"))

# Split train/test 80/20 in order to avoid data leakage
n = nrow(df)
n_train = round(Int, 0.8 * n)

train_df = df[1:n_train, :]
test_df  = df[(n_train+1):end, :]

# if column starts with ingredient_ , set as as target
feature_cols = filter(col -> !startswith(col, "ingredient_"), names(df))
target_cols = filter(col -> startswith(col, "ingredient_"), names(df))

X_train = Matrix(train_df[:, feature_cols])
X_test  = Matrix(test_df[:, feature_cols])

y_train = Matrix(train_df[:, target_cols])
y_test  = Matrix(test_df[:, target_cols])
;

In [14]:
# --- Sets ---
P = pizzas.pizza_id                     # pizza SKUs
I = ingredient_prices.ingredient        # ingredients

# --- Parameters ---

# Pizza demand (single day forecast)
d = Dict(p => daily_sales_actuals[!, p][1] for p in P)

# Pizza sale price
π = Dict(p => pizzas[!, :price][findfirst(==(p), pizzas[!, :pizza_id])] for p in P)

# Predicted ingredient availability (x_i is FIXED, not a decision variable)
x_pred = Dict(i => daily_sales_ingredients_predictions[!, i][1] for i in I)

# Ingredient usage per pizza (kg): r[p, i]
r = Dict()
for p in P
    pizza_idx = findfirst(==(p), pizzas[!, :pizza_id])
    pizza_type_id = pizzas[!, :pizza_type_id][pizza_idx]

    for i in I
        row_idx = findfirst(==(pizza_type_id), pizza_ingredient_matrix[!, :pizza_type_id])
        r[(p,i)] = pizza_ingredient_matrix[row_idx, i]
    end
end

# Initial ingredient inventory
I0 = Dict(i => 0.0 for i in I)

# Penalty for unmet pizza demand
α = Dict(p => 10.0 for p in P)

# Spoilage cost per kg of leftover ingredient
s = Dict(i => 0.1 * ingredient_prices[!, :price_per_kg][findfirst(==(i), ingredient_prices[!, :ingredient])] 
            for i in I)

# Storage capacity
C = 200.0
;


In [15]:
n_targets = length(target_cols)

# Function to expand dataset
function expand_dataset(X::Matrix, Y::Matrix)
    n_samples, n_features = size(X)
    _, n_targets = size(Y)

    # Prepare storage
    long_X = Float64[]
    long_y = Float64[]
    long_target_index = Int[]

    # Expand
    for i in 1:n_samples
        for j in 1:n_targets
            append!(long_X, vec(X[i, :]))     # feature columns
            push!(long_y, Y[i, j])            # target value
            push!(long_target_index, j)       # ingredient index
        end
    end

    # Reshape features
    long_X = reshape(long_X, n_features, n_samples*n_targets)'  # (rows, features)

    # One-hot encode target ingredient
    target_onehot_matrix = Array(onehotbatch(long_target_index, 1:n_targets))'  # (rows, n_targets)

    # Combine original features + one-hot
    X_long = hcat(long_X, target_onehot_matrix)
    y_long = long_y

    return X_long, y_long
end

# Expand train and test sets
X_train_long, y_train_long = expand_dataset(X_train, y_train)
X_test_long, y_test_long   = expand_dataset(X_test, y_test)

println("Original train size: ", size(X_train), " -> Expanded train size: ", size(X_train_long))
println("Original test size: ", size(X_test), " -> Expanded test size: ", size(X_test_long))

Original train size: (859, 549) -> Expanded train size: (56694, 615)
Original test size: (215, 549) -> Expanded test size: (14190, 615)


In [ ]:
grid = IAI.GridSearch(
    IAI.OptimalTreeRegressor(random_seed=123, max_categoric_levels_before_warning=50),
    max_depth=1:5,
    minbucket=1:10
)


IAI.fit!(grid, X_train_long, y_train_long)
best_model = IAI.get_learner(grid)


In [ ]:
# Predict on test
y_pred_long = IAI.predict(best_model, X_test_long)
y_pred_long = ceil.(y_pred_long)  # ceil to buy extra ingredient quantities

# Reshape predictions to original shape (n_samples, n_targets)
n_test_samples = size(X_test, 1)
y_pred_matrix = reshape(y_pred_long, n_targets, n_test_samples)'  # (rows=samples, cols=targets)

# Save predictions to CSV
pred_df = DataFrame(y_pred_matrix, Symbol.(target_cols))
CSV.write("point_predictions.csv", pred_df)


In [ ]:
# Leaf indices for train and test sets
leaf_train = IAI.apply(best_model, X_train_long)
leaf_test = IAI.apply(best_model, X_test_long)

# Similarity sets: for each test row (day), store indices of train rows in the same leaf
# Convert expanded indices back to original day indices
S = Dict{Int, Vector{Int}}()
n_train_samples = size(X_train, 1)

for t in 1:size(X_test, 1)
    # Find all expanded training rows in the same leaf as test day t
    expanded_indices = findall(leaf_train .== leaf_test[t])
    
    # Convert expanded indices to original day indices
    # Each original day is expanded into n_targets rows
    # Expanded index idx -> Original day = (idx - 1) ÷ n_targets + 1
    original_day_indices = unique([(idx - 1) ÷ n_targets + 1 for idx in expanded_indices])
    
    S[t] = original_day_indices
end

In [ ]:
function run_strategy_weighted(daily_sales_actuals, df_train; S, weight=true)
    total_days = size(daily_sales_actuals, 1)

    metrics = DataFrame(
        day = 1:total_days,
        pizza_produced = zeros(total_days),
        unmet_pizza_demand = zeros(total_days),
        leftover_ingredients = zeros(total_days),
        daily_profit = zeros(total_days)
    )

    total_profit = 0.0
    I0_current = Dict(i => 0.0 for i in I)

    for t in 1:total_days
        # --- Demand for pizzas today ---
        d_day = Dict(p => daily_sales_actuals[!, p][t] for p in P)

        # --- Weighted ingredient requirements ---
        req_i = Dict{eltype(I), Float64}()
        for i in I
            if weight && !isempty(S[t])
                # Weighted average across similar training rows
                req_i[i] = mean(df_train[S[t], i])
            else
                # Fallback: use mean across all train rows
                req_i[i] = mean(df_train[:, i])
            end
        end

        # Purchase only what you lack
        purchase_i = Dict(i => max(req_i[i] - I0_current[i], 0) for i in I)
        I_available = Dict(i => I0_current[i] + purchase_i[i] for i in I)

        # --- Solve production optimization ---
        model = Model(Gurobi.Optimizer)
        set_silent(model)

        @variable(model, y[p in P] >= 0, Int)
        @variable(model, z[p in P] >= 0, Int)
        @variable(model, w[i in I] >= 0)

        @objective(model, Max,
            sum(π[p] * y[p] for p in P)
            - sum(α[p] * z[p] for p in P)
            - sum(s[i] * w[i] for i in I)
            - sum(10*s[i] * purchase_i[i] for i in I)
        )

        @constraint(model, [p in P], y[p] + z[p] == d_day[p])
        @constraint(model, [i in I], sum(r[(p,i)]*y[p] for p in P) <= I_available[i])
        @constraint(model, [i in I], w[i] == I_available[i] - sum(r[(p,i)]*y[p] for p in P))

        optimize!(model)

        # Extract decisions
        y_opt = Dict(p => round(Int, value(y[p])) for p in P)
        z_opt = Dict(p => round(Int, value(z[p])) for p in P)
        w_opt = Dict(i => value(w[i]) for i in I)

        I0_current = deepcopy(w_opt)

        daily_profit = objective_value(model)
        total_profit += daily_profit

        metrics.pizza_produced[t] = sum(values(y_opt))
        metrics.unmet_pizza_demand[t] = sum(values(z_opt))
        metrics.leftover_ingredients[t] = sum(values(w_opt))
        metrics.daily_profit[t] = daily_profit
    end

    final_spoilage_cost = sum(s[i]*I0_current[i] for i in I)
    total_profit_net = total_profit - final_spoilage_cost

    return (total_profit = total_profit_net, metrics = metrics)
end


In [ ]:
# Remove ingredient_ prefix from target columns in train_df
for col in target_cols
    if col in names(train_df)
        rename!(train_df, col => Symbol(replace(String(col), "ingredient_" => "")))
    end
end

pred_result_weighted = run_strategy_weighted(daily_sales_actuals, train_df;
                                            S=S, weight=true)

println("Predictive weighted ORT total profit (net): ", pred_result_weighted.total_profit)